# Qwen3-1.7B Arabic Poetry Fine-Tuning with LoRA

This project fine-tunes the Qwen/Qwen3-1.7B large language model using Low-Rank Adaptation (LoRA) to generate Arabic poetry in response to user prompts.

## Setup
Install dependencies:
```bash
!pip install transformers
!pip install datasets
!pip install peft
!pip install accelerate
!pip install huggingface_hub
!pip install torch
```

## 📁 Dataset

The training data `(arabic_poetry_1000.json)` is a synthetic chat-style dataset containing 1000 samples of user prompts and poetic Arabic responses.

## Setup the model
### Libraries

In [34]:
import json
from datasets import Dataset
import torch
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from huggingface_hub import HfApi, get_token

from google.colab import userdata
hugging_token = userdata.get('HF_TOKEN')

## Step 1: Load and Preprocess the Dataset

In [ ]:
# Load the file manually
with open("/content/arabic_poetry_1000.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Convert the raw list of dictionaries to a HuggingFace Dataset object
dataset = Dataset.from_list(raw_data)

# Load tokenizer
model_name = "Qwen/Qwen3-1.7B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hugging_token)

- `apply_chat_template` formats the data into the structure expected by the model.

- `tokenize` adds padding and truncation, and sets labels = input_ids for CausalLM.

In [35]:
# Format the data using the chat template expected by the tokenizer
def format_chat(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["conversations"],
            tokenize=False,
            add_generation_prompt=False
        )
    }

# Tokenize the formatted text and prepare input_ids and labels for training

def tokenize(example):
    result = tokenizer(
        example["text"],             # Tokenize the formatted text
        truncation=True,             # Truncate text if longer than max_length
        padding="max_length",        # Pad text to fixed length
        max_length=1024,             # Set the maximum token length
    )
    result["labels"] = result["input_ids"].copy()  # Labels are the same as input_ids for causal LM
    return result

# First, format the dataset to add the 'text' column
dataset = dataset.map(format_chat, remove_columns=["conversations"])

# Then, tokenize the dataset using the newly created 'text' column
dataset = dataset.map(tokenize, remove_columns=["text"])

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    token=hugging_token
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

##  Step 2: LoRA Adapter Configuration

In [36]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##  Step 3: Training

In [5]:

# Define training hyperparameters and settings
training_args = TrainingArguments(
    output_dir="./qwen-poetry-lora2",           # Folder to save model checkpoints
    per_device_train_batch_size=2,              # Batch size per device (GPU)
    gradient_accumulation_steps=2,              # Accumulate gradients over 2 steps
    num_train_epochs=5,                         # Train for 5 full epochs
    learning_rate=2e-4,                         # Learning rate
    fp16=True,                                  # Use mixed precision training
    logging_steps=10,                           # Log training info every 10 steps
    save_steps=50,                              # Save checkpoint every 50 steps
    save_total_limit=2,                         # Keep only the last 2 checkpoints
    report_to="none",                           # Disable external logging (e.g., wandb)
    remove_unused_columns=False                 # Keep all columns (important for LoRA)
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,5.117200
20,3.397300
30,2.440900
40,1.829500
50,1.462800
60,1.174400
70,0.931200
80,0.736900
90,0.577700
100,0.473900


TrainOutput(global_step=1250, training_loss=0.3048844893455505, metrics={'train_runtime': 1544.484, 'train_samples_per_second': 3.237, 'train_steps_per_second': 0.809, 'total_flos': 4.33464016896e+16, 'train_loss': 0.3048844893455505, 'epoch': 5.0})

## Step 4: Save and Reload Model

In [6]:
model.save_pretrained("./qwen-poetry-arabic-lora")
tokenizer.save_pretrained("./qwen-poetry-arabic-lora")


('./qwen-poetry-lora2/tokenizer_config.json',
 './qwen-poetry-lora2/special_tokens_map.json',
 './qwen-poetry-lora2/chat_template.jinja',
 './qwen-poetry-lora2/vocab.json',
 './qwen-poetry-lora2/merges.txt',
 './qwen-poetry-lora2/added_tokens.json',
 './qwen-poetry-lora2/tokenizer.json')

## Step 5: Inference

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, "./qwen-poetry-arabic-lora")
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = tokenizer.apply_chat_template(
    [{"role": "user", "content": "اكتب لي بيت شعر عن النجاح."}],
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

user
اكتب لي بيت شعر عن النجاح.
assistant
<think>

</think>

سلامٌ على النجاح في كلِّ حينْ ** تزهو بهِ الأرواحُ كالبساتينْ


## Step 6: Upload to Hugging Face Hub

Make sure `hugging_token` has write access.

Adjust `user_name` to user name of huuging face
```python
repo_id = "user_name/qwen-poetry-arabic-lora"
```


In [25]:
repo_id = "mohammed-orabi2/qwen-poetry-arabic-lora"

# Create the repo (optional, safe if it exists)
api = HfApi()
api.create_repo(repo_id, exist_ok=True, token=hugging_token)

# Upload the entire folder
api.upload_folder(
    folder_path="qwen-poetry-arabic-lora",
    repo_id=repo_id,
    repo_type="model",
    token=hugging_token
)


adapter_model.safetensors:   0%|          | 0.00/6.44M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/12.9M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.44M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

Upload 16 LFS files:   0%|          | 0/16 [00:00<?, ?it/s]

...

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mohammed-orabi2/qwen-poetry-lora2/commit/36c146fac416d0a8c740f21b8ab5bb262ea42892', commit_message='Upload folder using huggingface_hub', commit_description='', oid='36c146fac416d0a8c740f21b8ab5bb262ea42892', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mohammed-orabi2/qwen-poetry-lora2', endpoint='https://huggingface.co', repo_type='model', repo_id='mohammed-orabi2/qwen-poetry-lora2'), pr_revision=None, pr_num=None)

## Step 7: Use from Anywhere

In [30]:
# Base model (must match your LoRA adapter)
base_model_id = "Qwen/Qwen3-1.7B"
adapter_id = "mohammed-orabi2/qwen-poetry-arabic-lora"

# Load tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load the LoRA adapter, providing the token
model = PeftModel.from_pretrained(base_model, adapter_id, token=hugging_token) # Pass the token here
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.44M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_fea

In [31]:
# Arabic prompt
prompt = "اكتب لي بيت شعر عن النجاح."

# Format it using Qwen chat template
chat = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(
    chat, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.95,
        temperature=0.9
    )

# Decode and print
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(response)


user
اكتب لي بيت شعر عن النجاح.
assistant
<think>

</think>

سلامٌ على النجاح في كلِّ حينْ ** تزهو بهِ الأرواحُ كالبساتينْ
